In [ ]:
# LoRA Streamlit UI: Quickstart for Windows
import time
import socket
import random
import os
import subprocess
import re
import shutil
import sys
import urllib.request
import zipfile

# Kill any previous Streamlit or cloudflared processes (safe in Windows)
def kill_process(process_name):
    try:
        subprocess.run(f'taskkill /F /IM {process_name}.exe', shell=True, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    except Exception as e:
        print(f"Could not kill {process_name}: {e}")

kill_process('streamlit')
kill_process('cloudflared')

# Wait a bit to ensure ports are released
time.sleep(2)

# Find a free port (prefer 8080, else pick 8081-8099)
def get_free_port(preferred=8080, fallback_range=(8081, 8099)):
    s = socket.socket()
    try:
        s.bind(("", preferred))
        port = preferred
    except OSError: 
        for p in range(*fallback_range):
            try:
                s.bind(("", p))
                port = p
                break
            except OSError:
                continue
        else:
            raise RuntimeError("No free port found in range 8080-8099")
    s.close()
    return port

PORT = get_free_port()
print(f"[INFO] Using port {PORT} for Streamlit and Cloudflared.")

# 1. Clone your repo (published repo URL)
if not os.path.exists('Lora_trainer'):
    subprocess.run(['git', 'clone', 'https://github.com/melbinjp/Lora_trainer.git'])
os.chdir('Lora_trainer')

# 1.5. Run preinstall_optimal_torch.py if present
if os.path.exists('preinstall_optimal_torch.py'):
    print('Running preinstall_optimal_torch.py...')
    subprocess.run([sys.executable, 'preinstall_optimal_torch.py', '--install'])
else:
    print('preinstall_optimal_torch.py not found, skipping.')

# 2. Run the smart setup script (Windows .bat)
subprocess.run(['cmd', '/c', 'setup.bat'])

# 3. Download cloudflared for Windows
cloudflared_url = 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-windows-amd64.exe'
cloudflared_path = os.path.abspath('cloudflared.exe')
if not os.path.exists(cloudflared_path):
    print('Downloading cloudflared for Windows...')
    urllib.request.urlretrieve(cloudflared_url, cloudflared_path)
    print('cloudflared downloaded.')

# 4. Start Streamlit in the background and wait for it to be ready
streamlit_proc = subprocess.Popen(
    [sys.executable, '-m', 'streamlit', 'run', 'test.py', f'--server.port={PORT}', '--server.enableCORS=false'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT
)

# Wait for Streamlit to start (look for 'You can now view your Streamlit app')
ready = False
for i in range(60):
    line = streamlit_proc.stdout.readline().decode(errors='ignore')
    print(line, end='')
    if 'You can now view your Streamlit app' in line:
        ready = True
        break
    time.sleep(1)
if not ready:
    print('Streamlit did not start in time. Check the logs above.')

# 5. Start cloudflared tunnel and print the public URL
if ready:
    tunnel = subprocess.Popen([cloudflared_path, 'tunnel', '--url', f'http://localhost:{PORT}'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    public_url = None
    for i in range(60):
        line = tunnel.stdout.readline().decode(errors='ignore')
        print(line, end='')
        if 'trycloudflare.com' in line:
            public_url = re.search(r'https://[\w\-]+\.trycloudflare.com', line)
            if public_url:
                print(f'\nYour public Streamlit URL: {public_url.group(0)}')
                break
        time.sleep(1)
    if not public_url:
        print('Cloudflared tunnel failed to start. Please check the output above or try restarting and running all cells again.')
else:
    print('Cloudflared tunnel not started because Streamlit did not start.')

# Note: The Streamlit UI will be available at the printed public URL above if successful.
# If the URL does not work, try restarting and running all cells again.